In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# set the url we want to visit
url = "https://dnr.wi.gov/lakes/CLMN/remotesensing/paths.aspx?county=Adams"

# visit that url, and grab the html of said page
html = requests.get(url)

In [3]:
# .text returns the request content in Unicode
html.text[:500]

'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n\r\n<head><meta http-equiv="content-type" content="text/html;charset=utf-8" /><link href="http://dnr.wi.gov/favicon.ico" type="image/x-icon" rel="icon" /><link href="http://dnr.wi.gov/favicon.ico" type="image/x-icon" rel="shortcut icon" /><title>\r\n\tRemote Sensing - Satellite Paths\r\n</title><!-- BEGIN global_head.inc ( /includes/global_hea'

In [4]:
# convert this into a soup object
soup = BeautifulSoup(html.text, 'html.parser')

In [6]:
from html_table_parser import parser_functions as parse


In [ ]:
table = soup.find('table',{'border':'1','bgcolor':'#aaaaaa'})

In [26]:
import pandas as pd
f = open('./data/report.csv','r')
lines = f.readlines()
f.close()
goodlines = []
start = 0
for i in range(len(lines)):
    if "Chlorophyll" in lines[i]:
        start = i
    if start > 0:
        if lines[i].strip().replace(",","") == "":
            goodlines = lines[start:i]
            break

f = open ("./data/out.csv","w")
f.writelines(goodlines)
f.close()
            
print( "\n".join(goodlines) )

Group Seq No, Start Date, Secchi (Feet),Secchi Hit Bottom?,Secchi (Meters),Chlorophyll(ug/l),Total Phosphorus(ug/l),Secchi TSI,Total Phosphorus TSI,Chlorophyll TSI,Lake Level,Staff Gauge,Appearance,Color,Perception

1,07/11/1994,3.3,,1,10.8,40,60,57,53,,,,,

1,08/01/1994,6.2,,1.9,5.15,19,51,51,47,,,,,

1,10/17/1994,5.2,,1.6,3.74,,53,,45,,,,,

18137361,04/07/2004,5,NO,1.5,,,54,,,NORMAL,,CLEAR,GREEN,1-Beautiful, could not be nicer

18137361,08/23/2004,5.5,NO,1.7,,,53,,,NORMAL,,CLEAR,BROWN,2-Very minor aesthetic problems

18137361,09/13/2004,5.5,NO,1.7,,,53,,,,,MURKY,BROWN,3-Enjoyment somewhat impaired (algae)

18137361,11/08/2004,6,YES,1.8,,,51,,,,,CLEAR,BROWN,2-Very minor aesthetic problems

18137361,02/10/2005,,,,,,,,,,,CLEAR,BROWN,2-Very minor aesthetic problems

18137361,04/15/2005,4.8,NO,1.5,,,55,,,,,MURKY,BROWN,2-Very minor aesthetic problems

18137361,06/27/2005,5,NO,1.5,,,54,,,,,MURKY,BROWN,2-Very minor aesthetic problems

18137361,07/18/2005,5.5,NO,1.7,,,53,,,,,MURKY,BROWN,3-Enj

In [27]:
df = pd.read_csv("./data/out.csv",sep=",")

ParserError: Error tokenizing data. C error: Expected 15 fields in line 5, saw 16
